In [1]:
# load packages and set paths

import dill
import scanpy as sc
import os
import warnings
warnings.filterwarnings("ignore")
import pandas
import pyranges
# Set stderr to null to avoid strange messages from ray
import sys
_stderr = sys.stderr
null = open(os.devnull,'wb')

# set paths
work_dir = '/Scottbrowne/members/smd/Projects/SD031/scenicplus/B16_CART'
tmp_dir = '/scratch2/devoes/tmp'

# set market
biomart_host = "http://nov2020.archive.ensembl.org/" # be sure to check for compatible capitalization nomenclature

# list of known TF
tf_file = "/Scottbrowne/members/smd/Projects/SD031/scenicplus/B16_CART/cistarget_database/HOCOMOCOv11_geneNames.txt" 

# load RNA AnnData, ATAC cistopic object, motif enrichment results
adata = sc.read_h5ad(os.path.join(work_dir, 'RNA/adata_SCA.h5ad'))
cistopic_obj = dill.load(open(os.path.join(work_dir, 'ATAC/cistopic_obj_48Topics.pkl'), 'rb'))
menr = dill.load(open(os.path.join(work_dir, 'motifs_no-promoter/menr.pkl'), 'rb'))

In [3]:
from scenicplus.scenicplus_class import create_SCENICPLUS_object
import numpy as np
scplus_obj = create_SCENICPLUS_object(
    GEX_anndata = adata.raw.to_adata(),
    cisTopic_obj = cistopic_obj,
    menr = menr,
    bc_transform_func = lambda x: f'{x}-GSE202543_B16_CART_tumor' #function to convert scATAC-seq barcodes to scRNA-seq ones
)
scplus_obj.X_EXP = np.array(scplus_obj.X_EXP.todense())
scplus_obj

2024-06-04 19:29:43,492 cisTopic     INFO     Imputing region accessibility
2024-06-04 19:29:43,494 cisTopic     INFO     Impute region accessibility for regions 0-20000
2024-06-04 19:29:44,813 cisTopic     INFO     Impute region accessibility for regions 20000-40000
2024-06-04 19:29:46,139 cisTopic     INFO     Impute region accessibility for regions 40000-60000
2024-06-04 19:29:47,458 cisTopic     INFO     Impute region accessibility for regions 60000-80000
2024-06-04 19:29:48,786 cisTopic     INFO     Impute region accessibility for regions 80000-100000
2024-06-04 19:29:50,125 cisTopic     INFO     Impute region accessibility for regions 100000-120000
2024-06-04 19:29:51,446 cisTopic     INFO     Impute region accessibility for regions 120000-140000
2024-06-04 19:29:52,784 cisTopic     INFO     Impute region accessibility for regions 140000-160000
2024-06-04 19:29:54,115 cisTopic     INFO     Impute region accessibility for regions 160000-180000
2024-06-04 19:29:55,443 cisTopic     

SCENIC+ object with n_cells x n_genes = 12295 x 17669 and n_cells x n_regions = 12295 x 312253
	metadata_regions:'Chromosome', 'Start', 'End', 'Width', 'cisTopic_nr_frag', 'cisTopic_log_nr_frag', 'cisTopic_nr_acc', 'cisTopic_log_nr_acc'
	metadata_genes:'gene_ids', 'feature_types', 'genome', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_cells'
	metadata_cell:'GEX_n_genes_by_counts', 'GEX_total_counts', 'GEX_total_counts_mt', 'GEX_pct_counts_mt', 'GEX_n_genes', 'GEX_doublet_score', 'GEX_predicted_doublet', 'GEX_leiden', 'ACC_Dupl_rate', 'ACC_Unique_nr_frag_in_regions', 'ACC_Unique_nr_frag', 'ACC_FRIP', 'ACC_TSS_enrichment', 'ACC_Total_nr_frag', 'ACC_Log_total_nr_frag', 'ACC_cisTopic_nr_frag', 'ACC_cisTopic_log_nr_frag', 'ACC_cisTopic_nr_acc', 'ACC_cisTopic_log_nr_acc', 'ACC_Log_unique_nr_frag', 'ACC_Total_nr_frag_in_regions', 'ACC_barcode', 'ACC_Dupl_nr_frag', 'ACC_n_genes_by_counts', 'ACC_total_counts', 'ACC_total_counts_mt', 'ACC_pct_counts_mt', 

In [4]:
from scenicplus.preprocessing.filtering import filter_genes,filter_regions
filter_genes(scplus_obj, min_pct = 0.5) # pct [0,100] 0.5% is must be in approx 65 cells
filter_regions(scplus_obj, min_pct = 0.5)

2024-06-04 19:30:10,102 Preprocessing INFO     Going from 17669 genes to 12080 genes.
2024-06-04 19:32:20,679 Preprocessing INFO     Going from 312253 regions to 242124 regions.


In [5]:
from scenicplus.cistromes import *
merge_cistromes(scplus_obj)

In [9]:
# define search space
from scenicplus.enhancer_to_gene import get_search_space, calculate_regions_to_genes_relationships, GBM_KWARGS
get_search_space(scplus_obj,
                 biomart_host = biomart_host,
                 species = 'mmusculus',
                 assembly = 'mm10',
                 extend_tss = [500,500],
                 remove_promoters = True,
                 upstream = [1000, 250000], # default is 150kb. I want to extend to 250kb
                 downstream = [1000, 250000])

2024-06-05 06:41:55,131 R2G          INFO     Downloading gene annotation from biomart dataset: mmusculus_gene_ensembl
2024-06-05 06:42:03,661 R2G          INFO     Downloading chromosome sizes from: http://hgdownload.cse.ucsc.edu/goldenPath/mm10/bigZips/mm10.chrom.sizes
2024-06-05 06:42:04,333 R2G          INFO     Extending promoter annotation to 500 bp upstream and 500 downstream
Warning! Start and End columns now have different dtypes: int32 and int64
Warning! Start and End columns now have different dtypes: int32 and int64
2024-06-05 06:42:05,320 R2G          INFO     Extending search space to:
            						250000 bp downstream of the end of the gene.
            						250000 bp upstream of the start of the gene.
Warning! Start and End columns now have different dtypes: int32 and int64
Warning! Start and End columns now have different dtypes: int32 and int64
2024-06-05 06:42:09,885 R2G          INFO     Intersecting with regions.


join: Strand data from other will be added as strand data to self.
If this is undesired use the flag apply_strand_suffix=False.
To turn off the warning set apply_strand_suffix to True or False.


Warning! Start and End columns now have different dtypes: int32 and int64
2024-06-05 06:42:11,311 R2G          INFO     Calculating distances from region to gene
2024-06-05 06:47:09,764 R2G          INFO     Removing DISTAL regions overlapping promoters
2024-06-05 06:47:20,735 R2G          INFO     Imploding multiple entries per region and gene
2024-06-05 06:57:08,364 R2G          INFO     Done!


In [11]:
# model
calculate_regions_to_genes_relationships(scplus_obj,
                    ray_n_cpu = 12,
                    _temp_dir = tmp_dir,
                    importance_scoring_method = 'GBM',
                    importance_scoring_kwargs = GBM_KWARGS) # imported from package

2024-06-05 07:04:16,334 R2G          INFO     Calculating region to gene importances, using GBM method


2024-06-05 07:04:21,731	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
Running using 12 cores: 100%|████████████████████████████████████████████████████| 10724/10724 [00:57<00:00, 187.39it/s]


2024-06-05 08:29:57,827 R2G          INFO     Took 5141.491611242294 seconds
2024-06-05 08:29:57,829 R2G          INFO     Calculating region to gene correlation, using SR method


2024-06-05 08:30:02,404	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
Running using 12 cores: 100%|████████████████████████████████████████████████████| 10724/10724 [00:54<00:00, 196.33it/s]


2024-06-05 09:34:10,918 R2G          INFO     Took 3853.0881016254425 seconds
2024-06-05 09:34:24,462 R2G          INFO     Done!


In [12]:
# Save
import pickle

with open(os.path.join(work_dir, 'scenicplus/scplus_obj_hocomocov11.pkl'), 'wb') as f:
  pickle.dump(scplus_obj, f)

In [15]:
from scenicplus.TF_to_gene import *
calculate_TFs_to_genes_relationships(scplus_obj,
                    tf_file = tf_file, # list of TF names
                    ray_n_cpu = 12,
                    method = 'GBM',
                    _temp_dir = tmp_dir, 
                    key= 'TF2G_adj')

2024-06-05 10:24:55,139	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


2024-06-05 10:24:56,135 TF2G         INFO     Calculating TF to gene correlation, using GBM method


Running using 12 cores: 100%|████████████████████████████████████████████████████| 12080/12080 [01:16<00:00, 158.09it/s]


2024-06-05 10:53:49,860 TF2G         INFO     Took 1733.723562002182 seconds
2024-06-05 10:53:49,862 TF2G         INFO     Adding correlation coefficients to adjacencies.
2024-06-05 10:54:23,292 TF2G         INFO     Warning: adding TFs as their own target to adjecencies matrix. Importance values will be max + 1e-05
2024-06-05 10:54:26,850 TF2G         INFO     Adding importance x rho scores to adjacencies.
2024-06-05 10:54:26,859 TF2G         INFO     Took 36.99719214439392 seconds


In [16]:
# Save
import pickle

with open(os.path.join(work_dir, 'scenicplus/scplus_obj_hocomocov11.pkl'), 'wb') as f:
  pickle.dump(scplus_obj, f)

In [22]:
from scenicplus.grn_builder.gsea_approach import build_grn

build_grn(scplus_obj,
         min_target_genes = 10,
         adj_pval_thr = 1,
         min_regions_per_gene = 0,
         quantiles = (0.85, 0.90), # 0.95?
         top_n_regionTogenes_per_gene = (5, 10, 15),
         top_n_regionTogenes_per_region = (),
         binarize_using_basc = True,
         rho_dichotomize_tf2g = True,
         rho_dichotomize_r2g = True,
         rho_dichotomize_eregulon = True,
         rho_threshold = 0.05,
         keep_extended_motif_annot = True,
         merge_eRegulons = True,
         order_regions_to_genes_by = 'importance',
         order_TFs_to_genes_by = 'importance',
         key_added = 'eRegulons_importance',
         cistromes_key = 'Unfiltered',
         disable_tqdm = True, #If running in notebook, set to True
         ray_n_cpu = 12,
         _temp_dir = tmp_dir)

2024-06-05 11:00:28,922 GSEA         INFO     Thresholding region to gene relationships


2024-06-05 11:00:32,735	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


2024-06-05 11:03:31,798 GSEA         INFO     Subsetting TF2G adjacencies for TF with motif.


2024-06-05 11:03:35,938	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


2024-06-05 11:03:36,852 GSEA         INFO     Running GSEA...


(_ray_run_gsea_for_e_module pid=3519) /home/devoes/miniconda3/envs/SCENICplus_SCA/lib/python3.10/site-packages/gseapy/algorithm.py:71: RuntimeWarning: divide by zero encountered in divide
(_ray_run_gsea_for_e_module pid=3519)   norm_tag =  1.0/sum_correl_tag
(_ray_run_gsea_for_e_module pid=3519) /home/devoes/miniconda3/envs/SCENICplus_SCA/lib/python3.10/site-packages/gseapy/algorithm.py:74: RuntimeWarning: invalid value encountered in multiply
(_ray_run_gsea_for_e_module pid=3519)   RES = np.cumsum(tag_indicator * correl_vector * norm_tag - no_tag_indicator * norm_no_tag, axis=axis)
(_ray_run_gsea_for_e_module pid=3523) /home/devoes/miniconda3/envs/SCENICplus_SCA/lib/python3.10/site-packages/gseapy/algorithm.py:71: RuntimeWarning: divide by zero encountered in divide [repeated 6x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.

2024-06-05 11:05:15,318 GSEA         INFO     Subsetting on adjusted pvalue: 1, minimal NES: 0 and minimal leading edge genes 10
2024-06-05 11:05:15,644 GSEA         INFO     Merging eRegulons
2024-06-05 11:05:15,700 GSEA         INFO     Storing eRegulons in .uns[eRegulons_importance].


In [23]:
import dill
with open(os.path.join(work_dir, 'scenicplus/scplus_obj_hocomocov11.pkl'), 'wb') as f:
  dill.dump(scplus_obj, f)